In [10]:
import urllib.request
import json
import os
import ssl
import base64
from PIL import Image
import io

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

def generate_and_display_image(prompt, api_key, url):
    """
    Generate an image using Azure ML endpoint and display it
    
    Args:
        prompt (str): Text prompt for image generation
        api_key (str): API key for Azure ML endpoint
        url (str): Azure ML endpoint URL
    """
    allowSelfSignedHttps(True)  # this line is needed if you use self-signed certificate

    # Prepare the request data
    data = {
        "input_data": {
            "data": [prompt],
            "columns": ["prompt"],
            "index": [0]
        }
    }

    # Encode the request body
    body = str.encode(json.dumps(data))

    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # Set up the request headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    # Make the request
    req = urllib.request.Request(url, body, headers)

    try:
        # Get the response
        response = urllib.request.urlopen(req)
        result = response.read()
        
        # Parse the JSON response
        response_dict = json.loads(result)
        
        print(response_dict)
        print(type(response_dict))
        
        # Assuming the response contains a base64 encoded image
        # The exact key might need to be adjusted based on your endpoint's response format
        image_data = response_dict[0].get('image_data') or response_dict[0].get('generated_image')
        
        if not image_data:
            print("No image data found in response")
            print("Full response:", response_dict)
            return

        # Decode base64 image
        image_bytes = base64.b64decode(image_data)
        
        # Create PIL Image
        image = Image.open(io.BytesIO(image_bytes))
        
        # Display the image
        image.show()
        
        # Optionally save the image
        image.save('generated_image.png')
        print("Image has been saved as 'generated_image.png'")
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code:", error.code)
        print("Headers:", error.info())
        print("Error response:", error.read().decode("utf8", 'ignore'))
    except Exception as e:
        print("An error occurred:", str(e))

# Example usage
if __name__ == "__main__":
    url = 'https://jnkazureproject-cbaij.eastus.inference.ml.azure.com/score'
    api_key = 'vyShhJf9grUnr0VNv6OzNnIxWeXkUDwq'  # Replace with your API key
    prompt = "A photograph of an astronaut riding a horse"
    
    generate_and_display_image(prompt, api_key, url)

[{'generated_image': '...', 'prompt': 'A photograph of an astronaut riding a horse', 'nsfw_content_detected': False}]
<class 'list'>
Image has been saved as 'generated_image.png'
